Define the seed for reproductible results

In [1]:
_SEED = 42

Read the dataset

In [2]:
import pandas as pd

df = pd.read_csv('../Output//Dataset_output/CP_TestingP@KDataset.csv', index_col=0)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,The sequence is sent over the network as follo...,Let s introduce the dynamics if on the prefix ...,Iterating over all possible values of and chec...,The main idea is that it is always better to r...,We cannot make a funny permutation only when b...,Note that any number in the sequence can be ma...,First of all let s delete all segments that do...,The solution of the problem can be found clear...,Since is odd exactly pieces of the board will ...,You are given the permutation f Let s build an...,Let s denote a prefix of length as We can note...
1,One day Polycarp decided to rewatch his absolu...,At first it doesn t matter if some season has ...,The main solution is dynamic programming let f...,Suppose there is no constraint on the number o...,This problem can be solved using the next gree...,Note the following fact if a number in a permu...,Let s first consider the possible paths across...,Let s first consider the possible paths across...,Find all segments of asterisks in the string L...,First let s note that we will have a transitio...,Let s declare a function which takes number as...
2,You are given a directed graph with vertices a...,Let s run dfs on the graph and color all back ...,To solve the problem we need to find the chara...,If was not really large then we could run the ...,Let the lowercase Latin letters be indexed fro...,Let be the minimum price of the berPhone durin...,If the edges were only added and not deleted i...,If there are two elements with the same value ...,Even though is up to straightforward solution ...,At first notice that if only programmers are t...,First let s consider the cases when the answer...
3,Let be an array of integers Let s define as a ...,As usual with tasks on an expected value let s...,Firstly if the graph consisting of initial ver...,First of all we will calculate the sum of maxi...,Let s intruduce the following dynamic programm...,In this problem it was enough to implement an ...,Solution is just some implementation simulate ...,We can process the cases of zero or one swap i...,The first and crucial observation is that we d...,Let s intruduce the following dynamic programm...,Let s consider each kilometer of the journey s...
4,Let s call an integer array if for each Let be...,Firstly let s learn the structure of good arra...,We can notice that the answer is the number of...,Let s run a dynamic programming from the leave...,There are two conditions that should be met ac...,The subsegment as well as the whole permutatio...,There are two main approaches to this problem ...,Let s intruduce the following dynamic programm...,In this problem we have to calculate the numbe...,There are many possible solutions One of them ...,If was not really large then we could run the ...


Define api request. Update the 'model' to desired gpt api (gpt-3.5-turbo or gpt-4-1106-preview)

In [3]:
from openai import OpenAI

def calculate_relevance(problem_statement, editorial, seed=None):
    context_promt = """
    You possess expertise in competitive programming, and your assistance is needed to score how relevant an editorial is for a problem statement.
    Answer a single line like this: "Relevance Score: X"
    """

    request_prompt = f"""Assign a relevance score to the editorial's suitability for the given problem statement on a scale from 0 to 10. Answer in JSON format respecting the pattern: "Relevance Score: X".
    PROBLEM STATEMNT: "{problem_statement}"
    EDITORIAL: "{editorial}"
    """

    client = OpenAI(
      api_key='API-KEY'
    )
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      # model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": context_promt},
        {"role": "user", "content": request_prompt}
      ],
      temperature=0.7,
      seed=seed,
      n=1,
      max_tokens=400
    )

    # Assuming `response` is your ChatCompletion object
    choice = response.choices[0]
    message_content = choice.message.content    

    return message_content

Define precision at k metric calculator

In [4]:
def precision_at_k(recommendations, k):
    """
    Computes precision at k for the given recommendations.
    Precision at k is defined as the ratio of relevant items in the top k recommendations to the total number of recommendations.

    Note: The target editorial has index 0
    Args:
        recommendations (list): The list of recommendations with associated scores.
        k (int): The value of k for which to compute the precision.

    Returns:
        int: 1 if the target editorial is in the top k recommendations, otherwise 0
    """
    assert recommendations is not None, "recommendations cannot be None"
    assert k is not None, "k cannot be None"

    # Take the top k recommendations
    top_k = [(recommendations[i], i) for i in range(len(recommendations))]

    # top_k.sort(key=lambda x: x[0])
    top_k.sort(key=lambda x: x[0], reverse=True)

    top_k = top_k[:k]

    # Calculate the number of relevant items in the top k recommendations
    editorial_pos = None
    for idx, editorial in enumerate(top_k):
        if editorial[1] == 0:
            editorial_pos = idx
            break

    if editorial_pos is not None:
        precision_at_k = 1
    else:
        precision_at_k = 0

    return precision_at_k

Call gpt api for the testing dataset

In [ ]:
import pandas as pd

LOWER_BOUND = 0
UPPER_BOUND = 309

# Initialize results_df as a DataFrame
results_df = pd.DataFrame()

for index, row in df.iloc[LOWER_BOUND:UPPER_BOUND].iterrows():
    problem_statement = row[0]
    editorials = row[1:]
    
    print("Row index: ", index)
    
    message_contents = []
    for editorial in editorials:
        message_content = calculate_relevance(problem_statement, editorial, seed=_SEED)
        if message_content is None:
            message_contents.append("""{"Relevance Score": 0}""")
        else:
            message_contents.append(message_content)

    # Convert the list to a DataFrame and append it to results_df
    message_df = pd.DataFrame([message_contents])
    results_df = pd.concat([results_df, message_df], ignore_index=True)
    
    if index % 50 == 0 and index != 0:
        results_df.to_excel(f'chat_gpt3_5_results_checkpoint_{index}.xlsx', index=False)

results_df.to_excel('chat_gpt3_5_results.xlsx', index=False)

P@K Score

In [6]:
import pandas as pd

results_df = pd.read_excel('chat_gpt3_5_results.xlsx')

In [7]:
precision_at_5_scores_cosine = []
precision_at_3_scores_cosine = []
precision_at_2_scores_cosine = []
precision_at_1_scores_cosine = []

In [8]:
import re


for index, row in results_df.iterrows():   
    similarity_scores_list = []
    for text in row:
        match = re.search(r'"?Relevance Score"?[:\s]*"?\b([\d\.]+)\b"?', text)
        if match:
            relevance_score = float(match.group(1))
            similarity_scores_list.append(relevance_score)
        else:
            print("No match!!")
            
    precision_at_5_scores_cosine.append(precision_at_k(similarity_scores_list, 5))
    precision_at_3_scores_cosine.append(precision_at_k(similarity_scores_list, 3))
    precision_at_2_scores_cosine.append(precision_at_k(similarity_scores_list, 2))
    precision_at_1_scores_cosine.append(precision_at_k(similarity_scores_list, 1))


In [9]:
output_scores = {
    'Mean Precision@5': sum(precision_at_5_scores_cosine) / len(precision_at_5_scores_cosine),
    'Mean Precision@3': sum(precision_at_3_scores_cosine) / len(precision_at_3_scores_cosine),
    'Mean Precision@2': sum(precision_at_2_scores_cosine) / len(precision_at_2_scores_cosine),
    'Mean Precision@1': sum(precision_at_1_scores_cosine) / len(precision_at_1_scores_cosine)
}

In [10]:
print(output_scores)


{'Mean Precision@5': 0.9902912621359223, 'Mean Precision@3': 0.9449838187702265, 'Mean Precision@2': 0.9029126213592233, 'Mean Precision@1': 0.7993527508090615}
